# 앙상블 학습 알고리즘

## 01. 랜덤 포레스트
- 결정트리를 랜덤하게 만들어 숲을 만듦

- 방법 : `from sklearn.ensemble import RandomForestClassifier`
```
부트스트랩 샘플 (랜덤하게 샘플을 뽑음. 한 샘플이 중복되어 추출될 수 있음.)   <--> 확률적 경사 하강법(모든 샘플이 소진될때까지 뽑음.)
특성 개수의 제곱근만큼의 특성 선택
```
    - 정형 데이터 : 표 형태 안에 숫자 데이터 ex) 엑셀, 데이터 베이스에 저장된 대부분의 데이터             --> `앙상블 학습 알고리즘` 활용 : 결정트리 기반
    - 비정형 데이터 : 사진, 텍스트 데이터, 음악      --> `트랜스 포머`

In [2]:
import pandas as pd
df = pd.read_csv('http://bit.ly/wine_csv_data')
data = df[['alcohol', 'sugar', 'pH']]             
target = df[['class']]

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(data, target)

In [5]:
from sklearn.model_selection import cross_validate     # 교차검증
from sklearn.ensemble import RandomForestClassifier    # 랜덤 포레스트

rf = RandomForestClassifier(n_jobs = -1)               # 기본적으로 100개의 결정트리 사용 -> n_jobs = -1로 모든 코어 사용
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)          

# 교차검증에서의 n_jobs = -1 : 병렬로 교차검증 수행  / return_train_score = 훈련세트에 대한 점수도 같이 반환 (과대적합 파악 용이) 

print(scores)

{'fit_time': array([0.28034997, 0.28043723, 0.27955985, 0.23342299, 0.23086047]), 'score_time': array([0.0325489 , 0.03036594, 0.0313518 , 0.02936363, 0.03011775]), 'test_score': array([0.89948718, 0.90564103, 0.89117043, 0.89835729, 0.88398357]), 'train_score': array([0.99923018, 0.99897357, 0.99897383, 0.99923037, 0.99897383])}


In [7]:
rf.fit(train_input, train_target)
rf.feature_importances_                 # feature_importances_ : 특성에 대한 중요도

C:\Users\1-17\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


array([0.23534942, 0.49796682, 0.26668376])

In [8]:
# oob : 부트스트랩 샘플링 시 선택되지 않은 샘플 (oob_score를 true로 하면 훈련 종료 후 oob 샘플을 기반으로 평가를 수행) : 검증세트의 역할 !

rf = RandomForestClassifier(oob_score = True, n_jobs = -1)
rf.fit(train_input, train_target)
rf.oob_score_                            

C:\Users\1-17\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.895935960591133

## 02. 엑스트라 트리
`from sklearn.ensemble import ExtraTreesClassifier`
- 랜덤 포레스트와 유사/ but, 속도가 빠름 (랜덤하게 노드 분할)

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1)

scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print(scores)

{'fit_time': array([0.18913865, 0.23476028, 0.19743085, 0.23226357, 0.25328422]), 'score_time': array([0.08549452, 0.04127955, 0.06786513, 0.05511403, 0.03542686]), 'test_score': array([0.89846154, 0.90461538, 0.88193018, 0.8963039 , 0.87474333]), 'train_score': array([0.99923018, 0.99897357, 0.99897383, 0.99923037, 0.99897383])}


## 03. 그레이디언트 부스팅
- 깊이가 얕은 결정트리 사용(결정트리를 추가하여 낮은 곳으로 이동)하여 이전 트리의 오차 보완
    결정 트리의 개수를 늘려도 --> 과대적합에 강함, 높은 일반화 성능 !
- `from sklearn.ensemble import GradientBoostingClassifier`

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=500)             # 결정 트리의 나무 갯수 : 500으로 지정 (결정트리 갯수 늘려도 과대적합 강함.)

scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print(scores)

{'fit_time': array([0.21275377, 0.24724197, 0.23041844, 0.18957758, 0.18705463]), 'score_time': array([0.0322392 , 0.0332036 , 0.03011274, 0.03048968, 0.03049183]), 'test_score': array([0.9025641 , 0.89435897, 0.88706366, 0.89835729, 0.87063655]), 'train_score': array([0.99923018, 0.99897357, 0.99897383, 0.99923037, 0.99897383])}


## 04. 히스토그램 기반 그레이디언트 부스팅
- `from sklearn.ensemble import HistGradientBoostingClassifier` : 그레이디언트 부스팅의 속도 개선
- 특성 중요도 : `from sklearn.inspection import permutation_importance`
    - 특성을 랜덤하게 섞은 이후에도 성능의 변화가 크지 않으면, 특성의 중요도가 낮은것 !

In [12]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier()

scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print(scores)

{'fit_time': array([0.21829057, 0.21684551, 0.26321077, 0.35464883, 0.26942444]), 'score_time': array([0.06974816, 0.09173393, 0.04579639, 0.03565121, 0.05961871]), 'test_score': array([0.89333333, 0.90461538, 0.87679671, 0.89938398, 0.87268994]), 'train_score': array([0.99923018, 0.99897357, 0.99897383, 0.99923037, 0.99897383])}


#### # 히스토그램 기반 그레이디언트 특성 중요도 : 특성을 랜덤하게 섞은뒤 모델의 성능이 변화하는지 확인 

In [18]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
scores = permutation_importance(hgb, train_input, train_target)

print(scores.importances_mean)


# 결과 해석 : 랜덤 포레스트와 비교하여 랜덤하게 섞었는데도 알코올, hp는 큰 변화가 없는것으로 보아 특성의 중요도 낮음.

C:\Users\1-17\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.09351396 0.24064039 0.08509852]
